In [9]:
# Imports

import carla
import math
import random
import time
import cv2
import sys
import numpy as np
import threading


In [10]:
# Setup

client = carla.Client('localhost', 2000)
world = client.get_world()

bp_lib = world.get_blueprint_library()
spawn_points = world.get_map().get_spawn_points()

car_bp = bp_lib.find('vehicle.lincoln.mkz_2020')
truck_bp = bp_lib.find('vehicle.carlamotors.firetruck')

spectator = world.get_spectator()

for actor in world.get_actors().filter('vehicle.*'):
    actor.destroy()


# Coordinates
x_car = 78.78953552246094
y_car = 218.3877716064453 
z_car = 0

dest_x_car = 238.78953552246094
dest_y_car = 177.3877716064453 
dest_z_car = z_car

x_truck = 109
y_truck = 160.6210479736328
z_truck = 0

dest_x_truck = 259
dest_y_truck = y_truck
dest_z_truck = z_truck

start_point_car = min(spawn_points, key=lambda spawn_point: spawn_point.location.distance(carla.Location(x_car, y_car, z_car)))
dest_point_car = min(spawn_points, key=lambda spawn_point: spawn_point.location.distance(carla.Location(dest_x_car, dest_y_car, dest_z_car)))

start_point_truck = min(spawn_points, key=lambda spawn_point: spawn_point.location.distance(carla.Location(x_truck, y_truck, z_truck)))
dest_point_truck = min(spawn_points, key=lambda spawn_point: spawn_point.location.distance(carla.Location(dest_x_truck, dest_y_truck, dest_z_truck)))

# Spawn the vehicle at the closest spawn point
car = world.try_spawn_actor(car_bp, start_point_car)
truck = world.try_spawn_actor(truck_bp, start_point_truck)





In [11]:

pref_speed = 50
speed_threshold = 2

def maintain_speed(s):
    if s >= pref_speed:
        return 0
    elif s < pref_speed - speed_threshold:
        return 0.8
    else:
        return 0.4

In [12]:

def angle_between(v1, v2):
    return math.degrees(np.arctan2(v1[1], v1[0]) - np.arctan2(v2[1], v2[0]))

def get_angle(vehicle, wp):
    vehicle_pos = vehicle.get_transform()
    vehicle_x = vehicle_pos.location.x
    vehicle_y = vehicle_pos.location.y
    wp_x = wp.transform.location.x
    wp_y = wp.transform.location.y

    x = (wp_x - vehicle_x) / ((wp_y - vehicle_y)**2 + (wp_x - vehicle_x)**2)**0.5
    y = (wp_y - vehicle_y) / ((wp_y - vehicle_y)**2 + (wp_x - vehicle_x)**2)**0.5

    vehicle_vector = vehicle_pos.get_forward_vector()
    degrees = angle_between((x,y), (vehicle_vector.x, vehicle_vector.y))

    return degrees

# Get information about vehicle
def get_info_about_vehicle(vehicle):
    vehicle_pos = vehicle.get_transform()
    vehicle_loc = vehicle_pos.location
    vehicle_rot = vehicle_pos.rotation
    print(f'{vehicle.id} location: ', vehicle_loc, 'Vehicle rotation: ', vehicle_rot)

# Get the size and draw a bounding box
def vehicle_bounding_box(vehicle, draw=False):
    bb = vehicle.bounding_box
    extent = bb.extent
    length = 2 * extent.x
    width = 2 * extent.y
    height = 2 * extent.z
    if draw:
        corners = [
            carla.Location(x=length/2, y=width/2, z=height),
            carla.Location(x=length/2, y=-width/2, z=height),
            carla.Location(x=length/2, y=-width/2, z=0),
            carla.Location(x=length/2, y=width/2, z=0),
            carla.Location(x=-length/2, y=width/2, z=height),
            carla.Location(x=-length/2, y=-width/2, z=height),
            carla.Location(x=-length/2, y=-width/2, z=0),
            carla.Location(x=-length/2, y=width/2, z=0),
        ]
        # Transform to world coordinates
        vehicle_pos = vehicle.get_transform()
        world_corners = [vehicle_pos.transform(location) for location in corners]

        color = carla.Color(r=0, g=0, b=255)
        life_time = 10
        persistent_lines = True
        thickness = 0.03

        # The bounding box does not currently follow the vehicle!
        
        # Draw the long lines
        for i in range(4):
            start = world_corners[i]
            end = world_corners[i + 4]
            vehicle.get_world().debug.draw_line(start, end, thickness=thickness, color=color, life_time=life_time, persistent_lines=persistent_lines)

        # Draw the front square
        for i in range(4):
            start = world_corners[i]
            end = world_corners[(i + 1) % 4]
            vehicle.get_world().debug.draw_line(start, end, thickness=thickness, color=color, life_time=life_time, persistent_lines=persistent_lines)

        # Draw the back square
        for i in range(4, 8):
            start = world_corners[i]
            end = world_corners[(i + 1) % 4 + 4]
            vehicle.get_world().debug.draw_line(start, end, thickness=thickness, color=color, life_time=life_time, persistent_lines=persistent_lines)

    print(f'Length: {length}, Width: {width}, Height: {height}')

In [13]:
# Test the angle function
# print(get_angle(car, route_car[0][0]))

In [14]:
# Route planner

def plan_route(world, start_point, dest_x, dest_y, dest_z, sampling_resolution=1, debug_draw=False):
    
    # This following line needs to be changed according to YOUR installation path
    carla_path = r'C:\Users\el00\OneDrive\Skrivbord\Carla 0.9.14\WindowsNoEditor\PythonAPI\carla'
    sys.path.append(carla_path)
    from agents.navigation.global_route_planner import GlobalRoutePlanner

    point_a = start_point.location
    point_b = carla.Location(x=dest_x, y=dest_y, z=dest_z)

    grp = GlobalRoutePlanner(world.get_map(), sampling_resolution)

    route = grp.trace_route(point_a, point_b)

    if debug_draw:
        for waypoint in route:
            world.debug.draw_string(
                waypoint[0].transform.location,
                '^',
                draw_shadow=False,
                color=carla.Color(r=0, g=0, b=255),
                life_time=60.0,
                persistent_lines=True
            )

    return route

# route_truck = plan_route(world, start_point_truck, dest_x_truck, dest_y_truck, dest_z_truck, sampling_resolution=1, debug_draw=True)
# route_car = plan_route(world, start_point_car, dest_x_car, dest_y_car, dest_z_car, sampling_resolution=1, debug_draw=True)

In [15]:
def drive_vehicle(vehicle, route, get_info = False):
    curr_wp = 0
    predicted_angle = get_angle(vehicle, route[0][0])

    while curr_wp < len(route) - 1:
        world.tick()

        while curr_wp < len(route) and vehicle.get_transform().location.distance(route[curr_wp][0].transform.location) < 5:
            curr_wp += 1

        predicted_angle = get_angle(vehicle, route[curr_wp][0])
        v = vehicle.get_velocity()
        speed = round(3.6 * math.sqrt(v.x**2 + v.y**2 + v.z**2), 0)
        estimated_throttle = maintain_speed(speed)

        if predicted_angle < - 300:
            predicted_angle = predicted_angle + 360
        elif predicted_angle > 300:
            predicted_angle = predicted_angle - 360
        steer_input = predicted_angle

        if predicted_angle < -40:
            steer_input = -40
        elif predicted_angle > 40:
            steer_input = 40
        
        steer_input = steer_input / 75

        vehicle.apply_control(carla.VehicleControl(throttle=estimated_throttle, steer=steer_input))

        if get_info:
            if curr_wp % 100 == 0:
                get_info_about_vehicle(vehicle)

    vehicle.apply_control(carla.VehicleControl(throttle=0, steer=0, brake=1))


In [16]:
# Test for running

route_truck = plan_route(world, start_point_truck, dest_x_truck, dest_y_truck, dest_z_truck, sampling_resolution=1, debug_draw=True)
route_car = plan_route(world, start_point_car, dest_x_car, dest_y_car, dest_z_car, sampling_resolution=1, debug_draw=True)

get_info = False
car_thread = threading.Thread(target=drive_vehicle, args=(car, route_car, get_info))
truck_thread = threading.Thread(target=drive_vehicle, args=(truck, route_truck, get_info))

# get_info_about_vehicle(car)
# get_info_about_vehicle(truck)
draw = True
vehicle_bounding_box(car, draw)
vehicle_bounding_box(truck, draw)

# # Start the threads
# car_thread.start()
# truck_thread.start()

# # Wait for both threads to finish
# car_thread.join()
# truck_thread.join()

Length: 4.89238166809082, Width: 1.8367133140563965, Height: 1.490277647972107
Length: 8.46804141998291, Width: 2.8910882472991943, Height: 3.8274123668670654
